<a href="https://colab.research.google.com/github/DavidP0011/etl/blob/main/app_video_transcribe_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title INICIALIZACIÓN

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verificar e instalar el paquete 'whisper'
try:
    import whisper
except ImportError:
    print("El paquete 'whisper' no está instalado. Procediendo con la instalación...")
    !pip install git+https://github.com/openai/whisper.git
    import whisper

# Verificar e instalar el paquete 'ffmpeg-python'
try:
    import ffmpeg
except ImportError:
    print("El paquete 'ffmpeg-python' no está instalado. Procediendo con la instalación...")
    !pip install ffmpeg-python

# Instalar y verificar 'ffmpeg' a nivel del sistema
import os
if os.system("ffmpeg -version") != 0:
    print("FFmpeg no está instalado. Procediendo con la instalación...")
    !apt-get update
    !apt-get install -y ffmpeg
    !ffmpeg -version
else:
    print("FFmpeg ya está instalado.")

import os
import time
import ffmpeg
import pandas as pd
from whisper import load_model
import gspread
from google.colab import auth
from google.auth.transport.requests import Request
from google.auth import default
import json
from datetime import datetime



Mounted at /content/drive
El paquete 'whisper' no está instalado. Procediendo con la instalación...
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-z9f07ucm
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-z9f07ucm
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803670 sha256=1fc31a92920cde83cf0616b4724682ee3574404c1e56ef9ba0cb44b71367be57
  Stored in directory: /tmp/pip-ephem-wheel-cache-qqecsa1e/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully

# DECLARACIÓN DE FUNCIONES

In [ ]:
# @title get_video_properties_dic()
def get_video_properties_dic(params: dict) -> dict:
    """
    Obtiene las propiedades técnicas de un archivo de video.
    Args:
        params (dict):
            - file_path (str): Ruta completa del archivo de video.
    Returns:
        dict: Diccionario con las siguientes claves:
            - file_name (str): Nombre del archivo de video.
            - file_path (str): Ruta completa del archivo de video.
            - file_size_mb (float): Tamaño del archivo en megabytes.
            - duration_s (float): Duración del video en segundos.
            - video_codec (str): Códec de video utilizado.
            - audio_codec (str): Códec de audio utilizado.
            - resolution (str): Resolución del video (ancho x alto).
            - audio_sample_rate (int): Frecuencia de muestreo del audio en Hz.
            - error (str): Mensaje de error en caso de fallo (opcional).
    """
    file_path = params.get("file_path")
    try:
        probe = ffmpeg.probe(file_path)
        video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
        audio_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'audio'), None)

        return {
            "file_name": os.path.basename(file_path),
            "file_path": file_path,
            "file_size_mb": float(probe['format']['size']) / (1024 * 1024),
            "duration_s": float(probe['format']['duration']),
            "video_codec": video_stream['codec_name'] if video_stream else None,
            "audio_codec": audio_stream['codec_name'] if audio_stream else None,
            "resolution": f"{video_stream['width']}x{video_stream['height']}" if video_stream else None,
            "audio_sample_rate": int(audio_stream['sample_rate']) if audio_stream else None,
        }
    except Exception as e:
        return {
            "file_name": os.path.basename(file_path),
            "file_path": file_path,
            "file_size_mb": None,
            "duration_s": None,
            "video_codec": None,
            "audio_codec": None,
            "resolution": None,
            "audio_sample_rate": None,
            "error": str(e)
        }

In [ ]:
# @title orchestrate_transcription_to_GSpread()
def orchestrate_transcription_to_GSpread(params: dict):
    """
    Orquesta el proceso de transcripción de videos en una carpeta y actualiza
    una hoja de Google Sheets con columnas fijas, incluyendo hasta 10 partes
    de transcripción (de 50k caracteres cada una).

    Escribe cada transcripción inmediatamente, para no perder avances si
    la sesión se interrumpe.

    Args:
        params (dict):
            - folder_path (str): Ruta de la carpeta con los videos.
            - model_size (str): Tamaño del modelo Whisper ("tiny", "base", "small", "medium", "large").
            - sheet_id (str | None): ID de la hoja de Google Sheets; si no se da, se crea una nueva.
            - verbose (bool): Mostrar mensajes detallados durante el proceso.
    """
    import gspread
    from google.colab import auth
    from google.auth.transport.requests import Request
    from google.auth import default
    from datetime import datetime
    import os
    import time
    import ffmpeg
    import pandas as pd
    from whisper import load_model
    from IPython.display import display

    # ------------------------------------------------------------------------
    # 1) Autenticar en Google y abrir/crear la hoja
    # ------------------------------------------------------------------------
    auth.authenticate_user()
    creds, _ = default()
    creds.refresh(Request())
    gspread_client = gspread.authorize(creds)

    sheet_id = params.get("sheet_id")
    if sheet_id:
        try:
            spreadsheet = gspread_client.open_by_key(sheet_id)
        except gspread.SpreadsheetNotFound:
            raise ValueError("El ID proporcionado no corresponde a una hoja válida.")
    else:
        folder_name = os.path.basename(params["folder_path"])
        sheet_name = "Transcripciones de Videos"
        spreadsheet = gspread_client.create(f"{folder_name} - {sheet_name}")

    sheet = spreadsheet.sheet1

    # ------------------------------------------------------------------------
    # 2) Definir cabecera fija y verificar si la hoja ya la tiene
    # ------------------------------------------------------------------------
    HEADERS = [
        "file_name", "file_path", "file_size_mb", "duration_s",
        "video_codec", "audio_codec", "resolution", "audio_sample_rate",
        "transcription_date",
        "transcription_part_1", "transcription_part_2", "transcription_part_3",
        "transcription_part_4", "transcription_part_5", "transcription_part_6",
        "transcription_part_7", "transcription_part_8", "transcription_part_9",
        "transcription_part_10"
    ]

    existing_values = sheet.get_all_values()
    if existing_values:
        current_headers = existing_values[0]
        # Chequeamos si 'file_name' no existe o si la cantidad de columnas
        # es distinta a la que esperamos (19), limpiamos y reescribimos.
        if ("file_name" not in current_headers) or (len(current_headers) != len(HEADERS)):
            sheet.clear()
            sheet.update("A1", [HEADERS])
    else:
        # Hoja en blanco
        sheet.update("A1", [HEADERS])

    # ------------------------------------------------------------------------
    # 3) Leer registros existentes para saber qué archivos se han transcrito
    # ------------------------------------------------------------------------
    all_rows = sheet.get_all_records()
    processed_files = {
        row.get("file_name"): row.get("transcription_date", "N/A")
        for row in all_rows
        if row.get("file_name") is not None
    }

    # ------------------------------------------------------------------------
    # 4) Listar archivos y mostrar estado
    # ------------------------------------------------------------------------
    folder_path = params["folder_path"]
    video_files = [
        f for f in os.listdir(folder_path)
        if f.lower().endswith((".mp4", ".mkv", ".avi", ".mov"))
    ]
    status_data = []
    for video in video_files:
        status = "Transcrito" if video in processed_files else "No transcrito"
        date = processed_files.get(video, "N/A")
        status_data.append({"file_name": video, "status": status, "transcription_date": date})

    status_df = pd.DataFrame(status_data)
    print("\nEstado de los vídeos:")
    display(status_df)

    # ------------------------------------------------------------------------
    # 5) Cargar modelo Whisper
    # ------------------------------------------------------------------------
    model = load_model(params.get("model_size", "medium"))
    if params.get("verbose", False):
        print(f"\nModelo '{params.get('model_size', 'medium')}' cargado correctamente.")

    # ------------------------------------------------------------------------
    # 6) Transcribir cada video y volcar de inmediato a la hoja
    # ------------------------------------------------------------------------
    max_chars = 50000  # troceamos la transcripción en bloques de 50k para evitar errores en GSheet

    for video in video_files:
        # Saltar si ya está transcrito
        if video in processed_files:
            if params.get("verbose", False):
                print(f"Saltando archivo ya procesado: {video}")
            continue

        video_path = os.path.join(folder_path, video)

        # Obtener metadatos del video (necesita get_video_properties_dic)
        properties = get_video_properties_dic({"file_path": video_path})

        # Transcribir
        transcription = ""
        try:
            if params.get("verbose", False):
                print(f"\nProcesando: {video}")
            result = model.transcribe(video_path, language='es')
            transcription = result["text"]
        except Exception as e:
            print(f"Error transcribiendo {video}: {e}")
            transcription = ""

        # Trocear en bloques de 50k
        transcription_parts = [
            transcription[i : i + max_chars]
            for i in range(0, len(transcription), max_chars)
        ]

        # Asignar un máximo de 10 partes
        if len(transcription_parts) > 10:
            # Concatenar el resto en la última parte
            sobrante = "".join(transcription_parts[10:])
            transcription_parts[9] += sobrante
            transcription_parts = transcription_parts[:10]

        # Preparar lista de 10 partes (rellenando con "")
        transcription_parts += [""] * (10 - len(transcription_parts))

        # Construir la fila en el orden de HEADERS
        row_values = [
            properties.get("file_name"),
            properties.get("file_path"),
            properties.get("file_size_mb"),
            properties.get("duration_s"),
            properties.get("video_codec"),
            properties.get("audio_codec"),
            properties.get("resolution"),
            properties.get("audio_sample_rate"),
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),   # transcription_date
        ] + transcription_parts  # las 10 columnas

        # Insertar la fila en la hoja
        reintentos = 3
        for intento in range(reintentos):
            try:
                sheet.append_row(row_values, value_input_option="USER_ENTERED")
                if params.get("verbose", False):
                    print(f"Actualizado en Google Sheets: {video}")
                break
            except gspread.exceptions.APIError as e:
                if intento < reintentos - 1:
                    print(f"Error al actualizar Google Sheets. Reintentando ({intento + 1}/{reintentos})...")
                    time.sleep(5)
                else:
                    print(f"Error persistente al actualizar Google Sheets: {e}")

    print(f"\nProceso completado. Hoja de cálculo disponible en: {spreadsheet.url}")


In [ ]:
#@title Configuración
import os
import time
import ffmpeg
import pandas as pd
from whisper import load_model

# Diccionario de configuración
folder_path = r"/content/drive/Shareddrives/AREA TIC COMPARTIDO USUARIOS EXTERNOS/ERP v6.0 - PROYECTO G-F (AYESA)/50 - BC/10 - DAF/Videos Sesiones DAF/" #@param {type:"string"}
model_size = "medium" #@param ["tiny", "base", "small", "medium", "large"]
sheet_id = "18esoAyRzemkA3GLVTDV6k45U5KqQzkrULYt3sE99KiQ" # o None
param = {
    "folder_path": folder_path,
    "model_size": model_size,
    "sheet_id": sheet_id,  # ID de la hoja de Google Sheets,
    "verbose": True  # Activar detalles adicionales
}


# Ejecutar el proceso orquestado
orchestrate_transcription_to_GSpread(param)


Estado de los vídeos:


,file_name,status,transcription_date
0,03 - Gestión Financiera 2-20240607_100241.mp4,No transcrito,N/A
1,03 - Gestión Financiera 2 20240607.mp4,No transcrito,N/A
2,01 - Ventas y Cobros-20240604_120138 - v1.0 20...,No transcrito,N/A
3,01 - Ventas y Cobros-20240604_100548 - v1.0 20...,No transcrito,N/A
4,07 - Sesión análisis Tesorería 2-20240618_...,No transcrito,N/A
5,06 - Sesión análisis Tesoreria 1-20240617_1...,No transcrito,N/A
6,05 - Compras y pagos 2-20240612_113249.mp4,No transcrito,N/A
7,05 - Compras y pagos 2-20240612_100906.mp4,No transcrito,N/A
8,04 - Compras y pagos-20240611_115156.mp4,No transcrito,N/A
9,04 - Compras y pagos-20240611_105212.mp4,No transcrito,N/A


100%|█████████████████████████████████████| 1.42G/1.42G [00:15<00:00, 95.8MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper


Modelo 'medium' cargado correctamente.

Procesando: 03 - Gestión Financiera 2-20240607_100241.mp4
Actualizado en Google Sheets: 03 - Gestión Financiera 2-20240607_100241.mp4

Procesando: 03 - Gestión Financiera 2 20240607.mp4
Actualizado en Google Sheets: 03 - Gestión Financiera 2 20240607.mp4

Procesando: 01 - Ventas y Cobros-20240604_120138 - v1.0 20240624.mp4
Actualizado en Google Sheets: 01 - Ventas y Cobros-20240604_120138 - v1.0 20240624.mp4

Procesando: 01 - Ventas y Cobros-20240604_100548 - v1.0 20240604.mp4
Actualizado en Google Sheets: 01 - Ventas y Cobros-20240604_100548 - v1.0 20240604.mp4

Procesando: 07 - Sesión análisis Tesorería  2-20240618_100526.mp4
Actualizado en Google Sheets: 07 - Sesión análisis Tesorería  2-20240618_100526.mp4

Procesando: 06 - Sesión análisis Tesoreria  1-20240617_100524.mp4
Actualizado en Google Sheets: 06 - Sesión análisis Tesoreria  1-20240617_100524.mp4

Procesando: 05 - Compras y pagos 2-20240612_113249.mp4
Actualizado en Goo